# Point & Shoot pothole reports via AI
This presents an AI model, trained for image classification in a [notebook format](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) accessible to those interested in learning more about adapting media uploads to leverage AI and location in photos. The demo directs your execution of code step-by-step, automatically interpreting photos as they upload to the cloud. In order to work with a photo, while on a phone, simply snap a photo and a webapp submits it to the AI - No more scrolling a long list of issue types, just point and shoot. For the purposes of this demo being done on a laptop however, accessing a photo needs to workaround the fact that on laptops, cameras function, integrate differently. Here we need a download plus upload step to proxy for a camera snapping a photo. With access to an image in this example, location is retreived from the image and,  AI determines issue-type with a model trained on photos of various issues:

1.   graffiti
2.   encampment
3.   garbage
4.   mural

are 4 categories covered in the trained AI model. This example shows classification on the 4 issue types as well as GPS and address interpretation using meta-data from the photo itself.
##A sample Photo
 Here is a graffiti photo, including the street number (2933). In lieu of a phone's camera (not part of this excercise) simulate camera activity with download/upload in this example. After you complete the cycle of running all the cells (download, upload , GPS, address, type classification ) on the provided photo, you may return to the **Upload a photo** step only this time you may select from your photos any that you took or that you want to interrogate location and or type classification. Just repeat the steps below on the new photo ( step 2 only has to be run a single time and can be omitted in subsequent cycles).

![sample 1](https://awsgcpupld-production.up.railway.app/pics/rclass_1.jpg)

##Instructions to complete this excercise
1. download this photo - right clik on it then "save image as.. "
2. run the cell installing EXIF component
3. UPLOAD to colab, the photo just downloaded by running cell "UPLOAD"
4. run the 2 cells below that to get latitude , longitude and Address
5. run image classification cell to get the type of photo and issue type


##Note on running a cell
Cells in Colab notebooks have icons in the far upper left of the cell. A mouseover on each icon will display details of instructions on the controls that you use within cells in a notebook. Once selected, runnable cells have a grey triangle icon in small black circle. Clik it to run a cell. When complete, observe the bottom row of the cell where output of the cell appears.

![run a cell](https://awsgcpupld-production.up.railway.app/pics/runColab.png)

##Install EXIF module

In [ ]:
# run this cell one-time to install software in CoLab sandbox
!pip install exif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 3.7 MB/s eta 0:00:00


##UPLOAD a photo for input in cells below
A file chooser like this one allows a choice:  
- image from your photoroll / gallery
- open camera to take new photo

The first time, for best results you can select the same downloaded image from above ( with graffiti ) as it is known to contain gps coordinates for the **photo gps** step below.
Once you have completed the cycle of notebook cells below, you may return to this cell and upload from whichever method you prefer. Remember the pathname of the file - used as input in subsequent steps in the notebook

In [ ]:
#select an input image using the button for "choose files"
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  fname = fn
  y=len(uploaded[fn])
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

if y>2950000: print('downscale this image before its classified using downscale cell')

Saving qa_1.jpg to qa_1.jpg
User uploaded file "qa_1.jpg" with length 1482965 bytes


In [ ]:
from PIL import Image

if y>2950000:
  print('file downsize image ')
  name,ext = os.path.splitext(fname)
  nwname = name + "-sm" + ext
  image = Image.open(fname)
  imgsml = image.resize((1000, 800))
  imgsml.save(nwname)

##Interpret the Photo's GPS
metadata of the photo includes latitude/ longitude that can be read using "EXIF" software. Prints out GPS coordinate values ( latitude, longitude )

In [ ]:
from exif import Image
# using the uploaded image above as input
# call  gps parse functions for latitude / longitude from image data
# 2 functions defined below are used by stmt at the bottom "image_coordinates"
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == 'S' or ref == 'W':
     decimal_degrees = -decimal_degrees
 return decimal_degrees

def image_coordinates(img_path):
    coords = (0,0)
    with open(img_path, 'rb') as src:
        img = Image(src)
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (decimal_coords(img.gps_latitude,
                      img.gps_latitude_ref),
                      decimal_coords(img.gps_longitude,
                      img.gps_longitude_ref))
        except AttributeError:
            print ('No Coordinates')
    else:
        print ('The Image has no EXIF information')
    return coords

# Step 1:  file from above to funtion that gets gps
respGps = image_coordinates(fname)
# format the response into variables ref'd by cell below
lat,long = respGps
print(f"lat: {lat}, long: {long}")


lat: 37.75251388888889, long: -122.41395555555556


##Google maps address  
 get a proper street address from the lat/ long coordinate values above by calling a service that implements the map feature (coordinate-to-street-address). Lists street address

In [ ]:
#if you do not have a google maps api key use this version
import requests
# a proxy to gogleapis.com hides the api-key. use the cell below if u have key
# Step 2: Construct the request and make the request
url = f'https://demo311-production.up.railway.app/addr/37.752756/-122.409781',



resp = requests.get(url).json()
json_res = resp
print(resp["data"])

InvalidSchema: ignored

##Roboflow API - image classification
A trained AI model inspects the photo, applying an issue-type label ( garbage, encampment, graffiti).
```
from roboflow import Roboflow

rf = Roboflow(api_key={API_KEY})
project = rf.workspace().project("org311-clip-photos")
model = project.version(2).model

# infer on a local image
print(model.predict("rclass_1.jpg").json())
```
The sample python code above from Roboflow samples performs a request for classification according to the trained, [org311-clip model](https://universe.roboflow.com/borneo/org311-clip-photos/model/2)

####Security Note

**API_KEY** in order to secure the key value, for the purpose of this demo, the code above is wrapped in a proxy. The actual call you will run in the cell below calls the proxy rather than directly creating a project and a model to execute the call.

Using a single line of code, the proxy securely appends the key, calling a REST endpoint for the image classification by Roboflow model.
```
await axios.post(`https://classify.roboflow.com/org311-clip-photos/2?api_key=${Config.api.ROBOFLOWKEY}`
```

 (docs) [Roboflow images](https://blog.roboflow.com/what-is-image-classification/)

In [ ]:
import requests

# Step 1: Read the file
filename = fname
with open(filename, "rb") as file:
    input = file.read()

# Step 2: Construct the request and make the request
url = "https://demo311-production.up.railway.app/imclass/photo.png"
headers = {"Content-Type": "application/octet-stream"}

resp = requests.post(url, data=input, headers=headers)
json_res = resp.json()
# print out the first 2 predictions for type of photo
ans = json_res["data"][0]
#resp["predictions"][0]["predictions"][0]
print(ans)
ans = json_res["data"][1]
print(ans)



{'class': 'garbage', 'confidence': 0.6178}
{'class': 'graffiti', 'confidence': 0.1363}


## Repeat, select a different image for input
Then rerun the cells above, starting with **Select Input image**
Take a photo or go to your photo roll and select an image you know to contain location ie was taken while your camera's settings included ( Android setting "use precise location", IOS  your own phot..
Process this new input photo by rerunning each cell that follows the select input cell.

The next notebook gets deeper into python features for interogating the image ( coordinates ), adding cloud storage for the photo ( AWS S3 ) and a issues database that can be queried using GPS methods such as **nearby**.